In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer,FreezeCoreTransformer

from qiskit.circuit.library import TwoLocal,EfficientSU2
from qiskit_nature.second_q.circuit.library.ansatzes import UCC,UCCSD
from qiskit_nature.second_q.circuit.library.initial_states import HartreeFock

from qiskit import Aer, IBMQ, QuantumCircuit
from qiskit.algorithms.optimizers import SPSA, L_BFGS_B, CG
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit_nature.runtime import VQEClient

from qiskit_nature.second_q.algorithms import GroundStateEigensolver, VQEUCCFactory
from qiskit.primitives import Estimator
estimator=Estimator()


In [ ]:
from custom_vqe import CustomVQEClient

In [ ]:
list=[]
for i in range (0,18):
    list.append(i)
for i in range (20,84):
    list.append(i)
print(list)

In [ ]:
driver = PySCFDriver(
    atom=f"Li -0.0000000000 0.0000000000 -3.1070698181 ; O -0.7347171102 -0.0000000000 -1.5079688529 ;O 0.7347171102 0.0000000000 -1.5079688529  ;Li 0.0000000000 0.0000000000 0.1515802566;O 0.6319228170 0.0000000000 2.1517114230 ;O -0.6319228170 -0.0000000000 2.1517114230 ",
    basis=" 6-31G(d,p) ",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
    )
# Electronic Structure problem is constructed by runing the driver
problem = driver.run()
# specify active space transformation
#transformer = ActiveSpaceTransformer(num_electrons=(1,1), num_spatial_orbitals=2,active_orbitals=[18,19])
transformer = FreezeCoreTransformer(freeze_core=True,remove_orbitals=list)
problem_reduced = transformer.transform(problem)
second_q_ops_reduced = problem_reduced.second_q_ops()
qubit_converter = QubitConverter(ParityMapper(), two_qubit_reduction=True)
num_spin_orbitals = problem_reduced.num_spin_orbitals
num_particles = problem_reduced.num_particles

qubit_op = qubit_converter.convert(second_q_ops_reduced[0], num_particles=problem_reduced.num_particles)
ansatz=UCC(num_spatial_orbitals=problem_reduced.num_spatial_orbitals , num_particles= num_particles,excitations='sd',qubit_converter=qubit_converter)

In [ ]:
ansatz.decompose().draw()

In [ ]:
init_state = HartreeFock(2, num_particles, qubit_converter)

In [ ]:
#Energy using VQEUCCFactory
vqe_factory = VQEUCCFactory(estimator,optimizer= CG(maxiter=500),ansatz= UCC(excitations='sd'))# This is an example of UCC"SD" ansatz
solver = GroundStateEigensolver(qubit_converter, vqe_factory)    
result = solver.solve(problem_reduced)  
min_energy = result.total_energies[0]
print(min_energy)

Using CustomVQEClient

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
#backend = provider.get_backend('ibmq_quito')
backend = provider.get_backend("simulator_statevector")

In [ ]:
from qiskit import IBMQ, QuantumCircuit, transpile
from qiskit.circuit import Parameter
qc_p = QuantumCircuit(2)
theta1=Parameter('θ1')
theta2=Parameter('θ2')
theta3=Parameter('θ3')
theta4=Parameter('θ4')
qc_p.ry(theta1,0)
qc_p.ry(theta2,1)
qc_p.h(1)
qc_p.cx(0, 1)
qc_p.h(1)
qc_p.ry(theta3,0)
qc_p.ry(theta4,1)
qc_p.draw()

In [ ]:
initial_point = np.random.random(qc_p.num_parameters)
optimizer= SPSA(maxiter=500)


In [ ]:
counts = []
values = []

def store(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

In [ ]:
runtime_vqe = CustomVQEClient(
                          ansatz=qc_p ,#ansatz,
                          optimizer=optimizer,
                          initial_point=initial_point,
                          provider=provider,
                          backend=backend,
                          shots=1024,
                           )

In [ ]:
runtime_vqe_groundstate_solver = GroundStateEigensolver(qubit_converter, runtime_vqe)
op,po=runtime_vqe_groundstate_solver.get_qubit_operators(problem_reduced)
joob=runtime_vqe.send_jobs(op,po)
print(joob)

In [ ]:
res1=runtime_vqe.compute_result(joob,problem_reduced,op,po)
Energy=res1.total_energies[0]
print(Energy)